In [1]:
import requests as r #In order to get the data from website

In [2]:
url='https://www.vins-etonnants.com/achat/cat-tous-les-vins-245.html?page=2'

In [3]:
headers="""'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
Accept-Encoding: gzip, deflate, br
Accept-Language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Cookie: _ga=GA1.2.383347107.1597591322; last_views=a%3A1%3A%7Bi%3A0%3Bi%3A9111%3B%7D; __atuvc=1%7C34; sid83ded6bc=ebr0cui8p13gqtth5tfnuk5qgifgc98n; _gid=GA1.2.195718740.1599739194
Host: www.vins-etonnants.com
Referer: https://www.vins-etonnants.com/achat/cat-tous-les-vins-245.html
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: same-origin
Sec-Fetch-User: ?1
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"""

In [4]:
headers=dict([i.strip().split(': ') for i in headers.split('\n')])

In [369]:
from bs4 import BeautifulSoup #In order to create soup of data
import time #In order to define break during the run
import re #In order to use regex

# First I create 4 list which will be fullfill during the loops below.
lst_url=[]
lst_soup=[]
lst_link=[]
lst_output=[]

# Second, I create a loop in order to obtain all url pages for the first 5 pages.
for i in range(5):
    resp=r.get(f'https://www.vins-etonnants.com/achat/cat-tous-les-vins-245.html?page={i}', headers=headers)
    html=resp.content
    soup=BeautifulSoup(html)
    soup=soup.select('.titre_produit_a')
    lst_soup.extend(soup)

# Third, I create a Loop over all the url in order to obtain every wine unique page links    
for i in lst_soup:
    lst_link.append(i.get('href'))

# Fourth, I create a loop over wine unique page links in order to retrieve all the data that I want:
for ix,i in enumerate(lst_link):
    
    # Soup1: retrieve Id, Name, Winemaker, Price
    soup1=BeautifulSoup(r.get(i).content)
    soup1=soup1.select('.titre_produit, h5, .fp_produit .prix span[property]')
    dict_output=(dict((i.get("property"), i.text.split(": ")[-1]) for i in soup1))
    
    # Soup2: retrieve type and descriptions of the wine
    soup2=BeautifulSoup(r.get(i).content)
    soup2=soup2.select('.fp_produit .vignettes .vignette')
    dict_output.update(dict((f'descr{ix}' , i.get('title')) for ix,i in enumerate (soup2)))
        
    #Soup3: retrieve region and grape variety
    soup3=BeautifulSoup(r.get(i).content)
    soup3=soup3.select('.description p')
    a=str([i.text for i in soup3])
    b={'Region':"".join(re.findall(r"(?<=\brégion\s)(\w+)", a)), 'Cépage':"".join(re.findall(r"(?<=\bCépage\s)(\w+)", a))}
    dict_output.update(b)
    
    # Soup 1, 2 and 3 are append in the lst_output
    lst_output.append(dict_output)
    
    # In order to do a break of 2sec every 5 loop. Always be nice with the machines!
    if ix%5==0: time.sleep(2)


In [378]:
import pandas as pd # In order to create a DataFrame

# Define a function to create the dataframe
def create_dataframe():
    df=pd.DataFrame(lst_output)
    return df

# Define a function to manipulate the columns (drop columns, rename columns and reindex columns)
def change_columns(df):
    df=df.drop(columns='gtin8')
    df.columns=['Name','Id','Winemaker','Price(€)','Descr0','Descr1','Region','Grape_variety','Descr2','Descr3','Descr4','Descr5']
    df=df.reindex(columns=['Id','Name','Winemaker','Region','Grape_variety','Price(€)','Descr0','Descr1','Descr2','Descr3','Descr4','Descr5'])
    return df

# Define a function to fill null values
def fill_na(df):
    df.Region.replace({"":"NC"}, inplace=True)
    df.Grape_variety.replace({"":"Multi grapes"}, inplace=True)

    df['Price(€)']=df['Price(€)'].fillna('Out of stock')
    df.Id=df.Id.fillna('Missing Id')
    df.Winemaker=df.Winemaker.fillna('Missing Winemaker')

    fill_cols=df.columns[df.isna().sum()>0]
    df[fill_cols]=df[fill_cols].fillna('N/A')
    return df

# Create a pipeline from all the functions defined above
create_dataframe().pipe(change_columns).pipe(fill_na)

,Id,Name,Winemaker,Region,Grape_variety,Price(€),Descr0,Descr1,Descr2,Descr3,Descr4,Descr5
0,tourcab,Ploussard La Cabane 2018,Touraize (Domaine de la),Jura,Poulsard,"21,90","Vin plus complexe, mais accessible à tous",Rouge peu tannique et fruité,N/A,N/A,N/A,N/A
1,tourmont,Chardonnay Montalaboz 2018,Touraize (Domaine de la),Jura,Chardonnay,"19,90","Vin plus complexe, mais accessible à tous",Blanc sec riche et structuré,N/A,N/A,N/A,N/A
2,margsap,Sapience 2010,Marguet (Champagne Benoit),NC,Multi grapes,"169,00",Vin classieux / excellent,Effervescent vineux,Vin bio,Sulfites très raisonnés,N/A,N/A
3,margcra,Les Crayères 2015,Marguet (Champagne Benoit),NC,Multi grapes,"85,50",Vin classieux / excellent,Effervescent vineux,Vin bio,Vin biodynamique,Sulfites très raisonnés,N/A
4,margav,Avize Cramant 2015,Marguet (Champagne Benoit),NC,Multi grapes,"85,50",Vin classieux / excellent,Effervescent vineux,Vin bio,Vin biodynamique,Sulfites très raisonnés,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
375,amabrut,Art Brut,Amadahy,NC,Multi grapes,Out of stock,Vin découverte/original,Rouge rond et fruité,Sans sulfites ajoutés,N/A,N/A,N/A
376,amayel,Yellow,Amadahy,NC,Multi grapes,Out of stock,Vin découverte/original,Vins oxydatifs,Sans sulfites ajoutés,N/A,N/A,N/A
377,berenfros,Champagne Enfant de la Montagne rosé Brut,Bertrand-Delespierre,NC,Multi grapes,"27,90","Vin plus complexe, mais accessible à tous",Effervescent vineux,N/A,N/A,N/A,N/A
378,crayalb,Chignin Bergeron Albinum 2018,Berlioz Adrien - Cellier des Cray,NC,Roussanne,Out of stock,"Vin plus complexe, mais accessible à tous",Blanc sec riche et structuré,Vin bio,N/A,N/A,N/A


In [348]:
#As we can see, there is no more null value after running the pipelines data cleaning functions
df.isna().sum()

Id               0
Name             0
Winemaker        0
Region           0
Grape_variety    0
Price(€)         0
Descr0           0
Descr1           0
Descr2           0
Descr3           0
Descr4           0
Descr5           0
dtype: int64